Для произвольно выбранного датасета провести обработку данных и построить предсказательную модель с использованием функционала pySpark.
Используем для выполнения задания датасет Walmart.

In [5]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, month, udf, lit, sum, to_date
from pyspark.sql.types import StringType, DoubleType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, StringIndexerModel, VectorAssembler, StandardScaler, PolynomialExpansion
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
spark.sparkContext.setLogLevel("ERROR")

In [6]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.spark import SparkTrials

In [7]:

# Создаем сессию Spark
spark = SparkSession.builder.appName("WalmartData").getOrCreate()

# Загрузка датасета
data = spark.read.csv('Walmart.csv', header=True, inferSchema=True)

# Просмотр первых строк датасета
data.show(5)

+-----+----------+------------+------------+-----------+----------+-----------+------------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
|    1|05-02-2010|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|
|    1|12-02-2010|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|
|    1|19-02-2010|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|
|    1|26-02-2010|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|
|    1|05-03-2010|  1554806.68|           0|       46.5|     2.625|211.3501429|       8.106|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
only showing top 5 rows



In [8]:
data.head()

Row(Store=1, Date='05-02-2010', Weekly_Sales=1643690.9, Holiday_Flag=0, Temperature=42.31, Fuel_Price=2.572, CPI=211.0963582, Unemployment=8.106)

In [9]:
data.dtypes

[('Store', 'int'),
 ('Date', 'string'),
 ('Weekly_Sales', 'double'),
 ('Holiday_Flag', 'int'),
 ('Temperature', 'double'),
 ('Fuel_Price', 'double'),
 ('CPI', 'double'),
 ('Unemployment', 'double')]

In [10]:

# Подсчет пропущенных значений в каждом столбце
missing_counts = data.select(
    [sum(col(column).isNull().cast("int")).alias(column) for column in data.columns]
)

# Вывод результата
missing_counts.show()

+-----+----+------------+------------+-----------+----------+---+------------+
|Store|Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|CPI|Unemployment|
+-----+----+------------+------------+-----------+----------+---+------------+
|    0|   0|           0|           0|          0|         0|  0|           0|
+-----+----+------------+------------+-----------+----------+---+------------+



Пропусков нет

In [11]:

# Преобразование столбца 'Date' в тип Date
data = data.withColumn('Date', to_date(col('Date'), 'dd-MM-yyyy'))

In [12]:
data.show(5)

+-----+----------+------------+------------+-----------+----------+-----------+------------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
|    1|2010-02-05|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|
|    1|2010-02-12|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|
|    1|2010-02-19|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|
|    1|2010-02-26|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|
|    1|2010-03-05|  1554806.68|           0|       46.5|     2.625|211.3501429|       8.106|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
only showing top 5 rows



In [13]:

# Определяем функцию для определения времени года
def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'autumn'

# Регистрируем UDF (пользовательскую функцию)
get_season_udf = udf(get_season, StringType())

# Добавляем колонку с временем года
data = data.withColumn('month', month(col('Date')))  # Извлекаем месяц из даты
data = data.withColumn('season', get_season_udf(col('month')))

# Указываем явный порядок времен года
seasons = ['winter', 'spring', 'summer', 'autumn']

# Создаем StringIndexerModel с явным порядком меток
indexer_model = StringIndexerModel.from_labels(
    labels=seasons,
    inputCol='season',
    outputCol='season_index',
    handleInvalid='keep'  # Обработка неизвестных значений
)

# Кодируем колонку 'season_index' в бинарные колонки
encoder = OneHotEncoder(inputCols=['season_index'], outputCols=['season_encoded'])

# Создаем пайплайн для выполнения преобразований
pipeline = Pipeline(stages=[indexer_model, encoder])
model = pipeline.fit(data)
data = model.transform(data)

# Функция для извлечения значения из вектора
def extract_value(vector, index):
    try:
        return float(vector[index])
    except:
        return 0.0

# Регистрируем UDF для извлечения значений
extract_value_udf = udf(extract_value, DoubleType())

# Создаем отдельные колонки для каждого времени года
for i, season in enumerate(seasons):
    data = data.withColumn(season, extract_value_udf(col('season_encoded'), lit(i)))

# Удаляем временные колонки
data = data.drop('Date','month', 'season', 'season_index', 'season_encoded')

# Просмотр результата
data.show()

[Stage 11:>                                                         (0 + 1) / 1]

+-----+------------+------------+-----------+----------+-----------+------------+------+------+------+------+
|Store|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|winter|spring|summer|autumn|
+-----+------------+------------+-----------+----------+-----------+------------+------+------+------+------+
|    1|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|   1.0|   0.0|   0.0|   0.0|
|    1|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|   1.0|   0.0|   0.0|   0.0|
|    1|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|   1.0|   0.0|   0.0|   0.0|
|    1|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|   1.0|   0.0|   0.0|   0.0|
|    1|  1554806.68|           0|       46.5|     2.625|211.3501429|       8.106|   0.0|   1.0|   0.0|   0.0|
|    1|  1439541.59|           0|      57.79|     2.667|211.3806429|       8.106|   0.0|   1.0|   0.0|   0.0|
|    1|  1

In [14]:

# Разделяем данные на тренировочную и тестовую выборки
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Разделяем train_data и test_data на X_train, X_test, y_train, y_test
X_train = train_data.drop('Weekly_Sales')
y_train = train_data.select('Weekly_Sales')

X_test = test_data.drop('Weekly_Sales')
y_test = test_data.select('Weekly_Sales')

In [15]:

# 1. Собираем все признаки в один вектор
assembler = VectorAssembler(
    inputCols=X_train.columns,  # Все колонки с признаками
    outputCol="features"
)

# 2. Масштабируем признаки
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withStd=True,
    withMean=True
)

# 3. Добавляем PolynomialFeatures (используем PolynomialExpansion)
poly_expansion = PolynomialExpansion(
    inputCol="scaled_features",
    outputCol="poly_features",
    degree=2  # Степень полинома (можно изменить)
)

# 4. Создаем модель Gradient Boosting
gbt = GBTRegressor(
    featuresCol="poly_features",  # Используем полиномиальные признаки
    labelCol="Weekly_Sales",     # Целевая переменная
    maxIter=100,                 # Количество итераций
    maxDepth=5,                  # Глубина дерева
    seed=42                      # Для воспроизводимости
)

# 5. Создаем пайплайн
pipeline = Pipeline(stages=[assembler, scaler, poly_expansion, gbt])

# 6. Обучаем модель
model = pipeline.fit(train_data)

# 7. Делаем предсказания на тестовых данных
predictions = model.transform(test_data)

# Просмотр результата
predictions.select("Weekly_Sales", "prediction").show(5)

+------------+------------------+
|Weekly_Sales|        prediction|
+------------+------------------+
|  1327405.42|1448272.6267190638|
|  1367320.01|1634289.3368308933|
|  1380020.27|1406605.6249667257|
|  1394393.84|1516647.5838208413|
|   1422711.6|1470002.8248235479|
+------------+------------------+
only showing top 5 rows



In [16]:
from pyspark.ml.evaluation import RegressionEvaluator

# Оценка модели
evaluator = RegressionEvaluator(
    labelCol="Weekly_Sales",
    predictionCol="prediction",
    metricName="rmse" 
)

rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 157320.68994475214


In [17]:
# Оценка модели
evaluator = RegressionEvaluator(
    labelCol="Weekly_Sales",
    predictionCol="prediction",
    metricName="r2" 
)

r2 = evaluator.evaluate(predictions)
print(f"R2: {r2}")

R2: 0.9237319137257813


В ходе работы выполнена предобработка данных и обучение модели с использованием PySpark.